<h1>Importálások<h1>

In [1]:
import functions as f

import cv2
import numpy as np

from skimage import feature
from itertools import chain

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.preprocessing import image_dataset_from_directory

from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split

import plotly.express as px
import pandas as pd

import joblib

<h1>1. Adatok beolvasása<h1>

<h4>Nem Deep learnin eljárások adata:<h4>

In [2]:
correct = f.make_data_from_folder("/Users/bormilan/Documents/plm_kepek/test/0")
incorrect_1 = f.make_data_from_folder("/Users/bormilan/Documents/plm_kepek/test/1")
incorrect_2 = f.make_data_from_folder("/Users/bormilan/Documents/plm_kepek/test/2")
incorrect_3 = f.make_data_from_folder("/Users/bormilan/Documents/plm_kepek/test/3")

In [3]:
positive_patches = f.apply_auto_brightness_and_contrast(correct, 10)
positive_patches = f.apply_gray(positive_patches)
positive_patches = np.stack(positive_patches, axis=0)

In [4]:
negative_1_patches = f.apply_auto_brightness_and_contrast(incorrect_1, 10)
negative_1_patches = f.apply_gray(negative_1_patches)
negative_1_patches = np.stack(negative_1_patches, axis=0)

In [5]:
negative_2_patches = f.apply_auto_brightness_and_contrast(incorrect_2, 10)
negative_2_patches = f.apply_gray(negative_2_patches)
negative_2_patches = np.stack(negative_2_patches, axis=0)

In [6]:
negative_3_patches = f.apply_auto_brightness_and_contrast(incorrect_3, 10)
negative_3_patches = f.apply_gray(negative_3_patches)
negative_3_patches = np.stack(negative_3_patches, axis=0)

<h2>Deep learning adatai:<h4>

In [7]:
ds_train_ = image_dataset_from_directory(
    "/Users/bormilan/Documents/plm_kepek_dl/train",
    labels='inferred',
    label_mode='categorical',
    image_size=[400, 400],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)
ds_valid_ = image_dataset_from_directory(
    "/Users/bormilan/Documents/plm_kepek_dl/valid",
    labels='inferred',
    label_mode='categorical',
    image_size=[400, 400],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)

ds_test_ = image_dataset_from_directory(
    '/Users/bormilan/Documents/plm_kepek_dl/test',
    labels='inferred',
    label_mode='categorical',
    image_size=[400, 400],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)

# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_test = (
    ds_test_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

Found 793 files belonging to 4 classes.
Found 218 files belonging to 4 classes.
Found 86 files belonging to 4 classes.


<h1>2. Data Preparation<h1>

<h4>HOG jellemzős reprezentációk elkészítése:<h4>

In [8]:
X_hog = np.array([feature.hog(im)
                    for im in chain(positive_patches,
                                    negative_1_patches,
                                    negative_2_patches,
                                    negative_3_patches)])
#pozitívak címkéi nullák
y_hog = np.zeros(X_hog.shape[0])
y_hog[: positive_patches.shape[0]] = 0

#első negatív halmaz címkéi egyesek
border_1 = positive_patches.shape[0] + negative_1_patches.shape[0]
y_hog[positive_patches.shape[0] : border_1] = 1

#második negatív halmaz címkéi kettesek
border_2 = border_1 + negative_2_patches.shape[0]
y_hog[border_1 : border_2] = 2

#harmadiké hármasok
y_hog[border_2 :] = 3

idx = np.random.permutation(len(X_hog))
X_hog,y_hog = X_hog[idx], y_hog[idx]


<h1>3. Modellek példányosítása<h1>

<h4>nem deep learning eljárások:<h4>

In [9]:
model_knn = KNeighborsClassifier(n_neighbors=1)
model_svc = SVC(kernel="linear")

# param_grid = {'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf','poly']}
# svc = SVC(probability=True)
# model_svc = GridSearchCV(svc,param_grid)


<h4>Deep learning modellek:<h4>

In [16]:
def createModel():
    model = Sequential()
    # The first two layers with 32 filters of window size 3x3
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=[400, 400, 3]))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation='softmax'))
    
    return model

<h1>4. Modellek tanítása<h1>

<h4>Nem Deep Learning<h4>

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_hog, y_hog, test_size=0.33, random_state=42)

In [12]:
model_knn.fit(X_train, y_train)
y_pred_knn = model_knn.predict(X_test)
knn_score = accuracy_score(y_test, y_pred_knn)

In [13]:
model_svc.fit(X_train, y_train)
y_pred_svc = model_svc.predict(X_test)
svc_score = accuracy_score(y_test, y_pred_svc)

In [14]:
print(knn_score)
print(svc_score)

0.9971988795518207
1.0


<h4>Deep Learning<h4>

In [18]:
model_cnn= createModel()

model_cnn.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model_cnn.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=10
)

Epoch 1/10
13/13 [==============================] - 192s 15s/step - loss: 1.7133 - accuracy: 0.2724 - val_loss: 1.3838 - val_accuracy: 0.2477
Epoch 2/10
13/13 [==============================] - 224s 17s/step - loss: 1.3881 - accuracy: 0.2661 - val_loss: 1.3842 - val_accuracy: 0.2890
Epoch 3/10
13/13 [==============================] - 202s 16s/step - loss: 1.3718 - accuracy: 0.3808 - val_loss: 1.3108 - val_accuracy: 0.6560
Epoch 4/10
13/13 [==============================] - 187s 14s/step - loss: 1.1649 - accuracy: 0.5460 - val_loss: 0.8431 - val_accuracy: 0.8578
Epoch 5/10
13/13 [==============================] - 210s 16s/step - loss: 0.5421 - accuracy: 0.8146 - val_loss: 0.2529 - val_accuracy: 0.9587
Epoch 6/10
13/13 [==============================] - 276s 22s/step - loss: 0.1796 - accuracy: 0.9458 - val_loss: 0.0311 - val_accuracy: 0.9862
Epoch 7/10
13/13 [==============================] - 197s 15s/step - loss: 0.1517 - accuracy: 0.9496 - val_loss: 0.0365 - val_accuracy: 0.9817
Epoch 

In [ ]:
cnn_score = model_cnn.evaluate(ds_test_)[1]

2/2 [==============================] - 1s 136ms/step - loss: 0.0000e+00 - accuracy: 1.0000


In [ ]:
df = pd.DataFrame(dict(
    scores = [knn_score, svc_score, cnn_score],
    models = ["knn", "SVC", "CNN"]
))
print([knn_score, svc_score, cnn_score])
fig = px.line(df, x="models", y="scores", title='Prediction Accuracy through different models.')
fig.update_yaxes()
fig.show()

NameError: name 'knn_score' is not defined

<h1>Modellek elmentése:<h1>

In [ ]:
filename = 'finalized_model_svc3.sav'
joblib.dump(model_svc, filename)

['finalized_model_svc3.sav']

In [ ]:
filename = 'finalized_model_knn3.sav'
joblib.dump(model_knn, filename)

['finalized_model_knn3.sav']

In [ ]:
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, y_test)
print(result)

In [1]:
#save the model
model_cnn.save("test_cnn_model8")

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model("test_cnn_model8") # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model in converted lite format
with open('test_cnn_model8/lite_cnn_model.tflite', 'wb') as f:
  f.write(tflite_model)

NameError: name 'model_cnn' is not defined

In [ ]:
tf.__version__

'2.7.0'

In [ ]:
reconstructed_model = keras.models.load_model("test_cnn_model3")
reconstructed_model.evaluate(ds_test_)[1]

5/5 [==============================] - 1s 216ms/step - loss: 0.0000e+00 - accuracy: 1.0000


1.0